In [28]:
import yfinance as yf
import pandas as pd
import re
import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, ConfusionMatrixDisplay
import seaborn as sns
import matplotlib
matplotlib.use('Agg')  # Use a non-interactive backend for script environments
import matplotlib.pyplot as plt
from gnews import GNews
from datetime import timedelta, date
from krazy import postgres_utilities as pu
from general import connect_to_db
from sqlalchemy.sql import text
from krazy.utility_functions import LoggerConfigurator
from krazy import postgres_utilities as pu
import numpy as np

engine = connect_to_db()
cur = engine.connect()

In [10]:
# initialize logging
logger = LoggerConfigurator('stock_analyzer',
                            r'C:\Users\dhaan\OneDrive\Documents\Python\AI Project\logs',
                            'DEBUG').configure()


2025-06-15 12:05:20.053 | DEBUG    | krazy.utility_functions:configure:1048 - Logger initialized at: C:\Users\dhaan\OneDrive\Documents\Python\AI Project\logs\stock_analyzer.log


In [11]:

# Parameters
TICKER = "AAPL"
START_DATE = "2025-04-01"  
END_DATE = "2025-06-12"     # Keep end date as before
QUERY = "Apple"
GNEWS_API_KEY = "abe68bc5c29ee58b31c7ebbcb07b1290"  # Updated API key


In [15]:

# 1. Fetch stock data
def stock_price_fetch_data(ticker, stock_name, start_date, end_date):
    df_price = yf.download(ticker, start=start_date, end=end_date)
    df_price['runtime'] = datetime.datetime.now()
    if isinstance(df_price.columns, pd.MultiIndex):
        df_price.columns = df_price.columns.get_level_values(0)
    df_price['pct_change'] = df_price['Close'].pct_change() * 100
    df_price.reset_index(inplace=True)
    df_price['date']  = pd.to_datetime(df_price['Date']).dt.date
    df_price['stock_name']=stock_name
    df_price['stock_code']=ticker
    logger.debug('Stock prices fectched successfully')
    return df_price

df_p = stock_price_fetch_data('AAPL','Apple', '2025-04-01', '2025-06-12')
# df_p


C:\Users\dhaan\AppData\Local\Temp\ipykernel_7288\3162635685.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_price = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed
2025-06-15 12:10:40.405 | DEBUG    | __main__:stock_price_fetch_data:12 - Stock prices fectched successfully


In [16]:
def stock_price_push_data(df_price):
    
    df_price['unique_key'] = df_price['Date'].astype(str)
    pu.dbase_writer_dup_handled(
        engine,
        df_price,
        'stock_analyzer',
        'stock_prices',
        'unique_key',
        files_processed=None,
        update_dup=True
    )
    # df_price.to_sql('stock_prices', con=engine, if_exists='append', index=False)
    logger.debug('Stock prices pushed to database successfully')
    
stock_price_push_data(df_p)

length of data: 50
Updating 0 existing records in database
Pushing 50 records to database


2025-06-15 12:10:47.358 | DEBUG    | __main__:stock_price_push_data:14 - Stock prices pushed to database successfully


New columns added: []
Pushed 50 records to database


In [8]:
data = cur.execute(text('select * from stock_analyzer.stock_data;')).fetchall()
print(data)


KeyboardInterrupt



In [ ]:
# cur.rollback()

In [ ]:
# sql_queries = ['Alter table stock_analyzer.stock_data add column if not exists test varchar(10);']

# for sql_query in sql_queries:
#     print(sql_query)
#     try:
#         cur.execute(text(sql_query))
#         cur.commit()
#     except Exception as e:
#         print(f"Error: {e}")
#         cur.rollback()

Alter table stock_analyzer.stock_data add column if not exists test varchar(10);


In [ ]:
# df = pd.read_sql_query('select * from stock_analyzer.stock_data;', engine)
# df.head()

,Date,Close,High,Low,Open,Volume,pct_change,date,test
0,2023-06-12,181.968094,182.067109,179.176056,179.473085,54274900,NaN,2023-06-12,None
1,2023-06-13,181.492859,182.324528,180.631488,180.987920,54929100,-0.261164,2023-06-13,None
2,2023-06-14,182.126495,182.562136,180.215635,181.552243,57462900,0.349125,2023-06-14,None
3,2023-06-15,184.166092,184.671046,181.958202,182.136425,65433200,1.119879,2023-06-15,None
4,2023-06-16,183.086868,185.136355,182.443318,184.878923,101235600,-0.586006,2023-06-16,None


In [24]:
def all_articles_fetch_data(query, start_dt, end_dt):
    
    all_articles = []

    def daterange(start_dt, end_dt, step_days=30):
        for n in range(0, (end_dt - start_dt).days, step_days):
            yield start_dt + timedelta(n), min(start_dt + timedelta(n + step_days), end_dt)

    start_dt = pd.to_datetime(start_dt).date()
    end_dt = pd.to_datetime(end_dt).date()

    for start, end in daterange(start_dt, end_dt, step_days=30):
        gnews = GNews(language='en', max_results=100, start_date=start, end_date=end)
        articles = gnews.get_news(query)
        # for article in articles:
        #     published = article.get('published date')
        #     if hasattr(published, 'date'):
        #         art_date = published.date()
        #     elif isinstance(published, str):
        #         try:
        #             art_date = pd.to_datetime(published).date()
        #         except Exception:
        #             art_date = None
        #     else:
        #         art_date = None
            # if art_date and (start <= art_date <= end):
        all_articles.extend(articles)
    all_articles = pd.DataFrame(all_articles)
    all_articles.rename(columns={'title':'headline'}, inplace=True)
    all_articles['link'] = all_articles['publisher'].apply(lambda x: dict(x).get('href'))
    all_articles['title'] = all_articles['publisher'].apply(lambda x: dict(x).get('title'))
    all_articles['published date'] = pd.to_datetime(all_articles['published date'])
    all_articles['date'] = all_articles['published date'].dt.date
    all_articles['stock_code'] = query
    all_articles.drop(columns=['publisher'], inplace=True)
        
    logger.debug('All articles fetched successfully')
    return all_articles
    
all_articles = all_articles_fetch_data(TICKER, START_DATE, END_DATE)


2025-06-15 12:50:34.356 | DEBUG    | __main__:all_articles_fetch_data:37 - All articles fetched successfully


In [18]:
all_articles

,headline,description,published date,url,link,title,date
0,Understanding Aggregate Trends for Apple Intel...,Understanding Aggregate Trends for Apple Intel...,2025-04-14 07:00:00,https://news.google.com/rss/articles/CBMihwFBV...,https://machinelearning.apple.com,Apple Machine Learning Research,2025-04-14
1,Sleeping with the Apple Watch - Six Colors,Sleeping with the Apple Watch Six Colors,2025-04-23 07:00:00,https://news.google.com/rss/articles/CBMic0FVX...,https://sixcolors.com,Six Colors,2025-04-23
2,"“Vibes Are Great, Honestly”: Let’s Meet Apple ...","“Vibes Are Great, Honestly”: Let’s Meet Apple ...",2025-04-02 07:00:00,https://news.google.com/rss/articles/CBMickFVX...,https://www.interviewmagazine.com,Interview Magazine,2025-04-02
3,Apple iPhone 16e Smartphone Review - CGMagazine,Apple iPhone 16e Smartphone Review CGMagazine,2025-04-05 07:00:00,https://news.google.com/rss/articles/CBMifEFVX...,https://www.cgmagonline.com,CGMagazine,2025-04-05
4,"MSU, Apple collaborate to open manufacturing a...","MSU, Apple collaborate to open manufacturing a...",2025-04-03 07:00:00,https://news.google.com/rss/articles/CBMidEFVX...,https://msutoday.msu.edu,MSUToday,2025-04-03
...,...,...,...,...,...,...,...
195,"Apple blocks Fortnite game on iPhones, video g...","Apple blocks Fortnite game on iPhones, video g...",2025-05-16 07:00:00,https://news.google.com/rss/articles/CBMib0FVX...,https://www.cbsnews.com,CBS News,2025-05-16
196,Why Apple Still Hasn’t Cracked AI - Bloomberg.com,Why Apple Still Hasn’t Cracked AI Bloomberg.com,2025-05-18 07:00:00,https://news.google.com/rss/articles/CBMioAFBV...,https://www.bloomberg.com,Bloomberg.com,2025-05-18
197,Apple’s court loss to Epic Games is a stunning...,Apple’s court loss to Epic Games is a stunning...,2025-05-02 07:00:00,https://news.google.com/rss/articles/CBMiogFBV...,https://venturebeat.com,VentureBeat,2025-05-02
198,"Apple considers raising iPhone prices, WSJ rep...","Apple considers raising iPhone prices, WSJ rep...",2025-05-12 07:00:00,https://news.google.com/rss/articles/CBMimgFBV...,https://www.reuters.com,Reuters,2025-05-12


In [33]:
def sentiment_analysis(ticker:str, start_date:str, end_date:str)->bool:
    df_articles = pd.read_sql_query(f'''select * 
                    from 
                        stock_analyzer.stock_articles sa 
                    where 
                        sa.stock_code ~* '{ticker}' 
                        and sa."date" between '{start_date}' 
                        and '{end_date}' 
                        and sa.sentiment_score is null;
                    ''', engine)
    # do sentimant analysis on df_articles and save the score in column 'sentiment_score'
    analyzer = SentimentIntensityAnalyzer()
    df_articles['sentiment_score'] = df_articles['headline'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
    try:
        pu.dbase_writer_dup_handled(
            engine,
            df_articles[['sentiment_score', 'row_id']],
            'stock_analyzer',
            'stock_articles',
            'row_id',
            files_processed=None,
            update_dup=True
        )
        return True
    except Exception as e:
        logger.error(f"Error in pushing sentiment score: {e}")
        return False
    
result = sentiment_analysis(TICKER, START_DATE, END_DATE)
print(f"Sentiment analysis result: {result}")

length of data: 300
Updating 300 existing records in database
No new records to push to database
Sentiment analysis result: True


In [25]:
def all_articles_push_data(all_articles):
    
    all_articles['unique_key'] = all_articles['date'].astype(str) 
    all_articles['sentiment_score'] = np.nan
    pu.dbase_col_checker_adder("stock_analyzer", 'stock_articles', all_articles, engine, speak=True)
    pu.dbase_writer_dup_handled(
        engine,
        all_articles,
        'stock_analyzer',
        'stock_articles',
        'unique_key',
        files_processed=None,
        update_dup=True
    )
    logger.debug('All articles pushed to database successfully')
    
all_articles_push_data(all_articles)

Table 'stock_articles' does not exist.
length of data: 300
Updating 0 existing records in database
Pushing 300 records to database
New columns added: []


2025-06-15 12:51:05.893 | DEBUG    | __main__:all_articles_push_data:14 - All articles pushed to database successfully


Pushed 300 records to database


In [37]:
def fetch_prices(ticker:str, start_date:str, end_date:str)->pd.DataFrame:
    df = pd.read_sql_query(f'''select * from stock_analyzer.stock_prices sp where sp.stock_code ~* 
                           '{ticker}' and sp."date" between '{start_date}' and '{end_date}';''', engine)    
    return df

In [47]:
df_price = fetch_prices(TICKER, START_DATE, END_DATE)
df_price

,Date,Close,High,Low,Open,Volume,runtime,pct_change,date,stock_name,stock_code,unique_key,row_id
0,2025-04-01,222.897705,223.387054,218.613315,219.522127,36412700,2025-06-15 12:10:40.398107,NaN,2025-04-01,Apple,AAPL,2025-04-01,1
1,2025-04-02,223.596786,224.895087,220.730550,221.030160,35905900,2025-06-15 12:10:40.398107,0.313633,2025-04-02,Apple,AAPL,2025-04-02,2
2,2025-04-03,202.923904,207.218276,200.986443,205.270818,103419000,2025-06-15 12:10:40.398107,-9.245608,2025-04-03,Apple,AAPL,2025-04-03,3
3,2025-04-04,188.133301,199.618240,187.094654,193.636079,125910900,2025-06-15 12:10:40.398107,-7.288744,2025-04-04,Apple,AAPL,2025-04-04,4
4,2025-04-07,181.222366,193.895735,174.391312,176.967935,160466300,2025-06-15 12:10:40.398107,-3.673424,2025-04-07,Apple,AAPL,2025-04-07,5
5,2025-04-08,172.194199,190.090729,168.988411,186.455496,120859500,2025-06-15 12:10:40.398107,-4.981818,2025-04-08,Apple,AAPL,2025-04-08,6
6,2025-04-09,198.589584,200.347274,171.664886,171.724805,184395900,2025-06-15 12:10:40.398107,15.328847,2025-04-09,Apple,AAPL,2025-04-09,7
7,2025-04-10,190.170624,194.524915,182.760343,188.822401,121880000,2025-06-15 12:10:40.398107,-4.239377,2025-04-10,Apple,AAPL,2025-04-10,8
8,2025-04-11,197.890488,199.278667,185.816325,185.856281,87435900,2025-06-15 12:10:40.398107,4.059441,2025-04-11,Apple,AAPL,2025-04-11,9
9,2025-04-14,202.254776,212.661128,200.896557,211.163092,101352900,2025-06-15 12:10:40.398107,2.205406,2025-04-14,Apple,AAPL,2025-04-14,10


In [27]:
def articles_fetch_data(ticker:str, start_date:str, end_date:str)->pd.DataFrame:
    df_articles = pd.read_sql_query(f'''select * from stock_analyzer.stock_articles sa 
                                    where sa.stock_code ~* '{ticker}' 
                                    and sa."date" between '{start_date}' and '{end_date}';''', engine)
    return df_articles

df_articles = articles_fetch_data('AAPL', '2025-04-01', '2025-05-12')
df_articles

,headline,description,published date,url,link,title,date,stock_code,unique_key,row_id
0,Is Apple Inc. (NASDAQ:AAPL) the Best NASDAQ St...,Is Apple Inc. (NASDAQ:AAPL) the Best NASDAQ St...,2025-04-16,https://news.google.com/rss/articles/CBMifEFVX...,https://finance.yahoo.com,Yahoo Finance,2025-04-16,AAPL,2025-04-16,1
1,"Going Into Earnings, Is Apple Stock a Buy, a S...","Going Into Earnings, Is Apple Stock a Buy, a S...",2025-04-25,https://news.google.com/rss/articles/CBMingFBV...,https://www.morningstar.com,Morningstar,2025-04-25,AAPL,2025-04-25,2
2,Apple (AAPL) Stock Moves -1.94%: What You Shou...,Apple (AAPL) Stock Moves -1.94%: What You Shou...,2025-04-21,https://news.google.com/rss/articles/CBMihgFBV...,https://www.nasdaq.com,Nasdaq,2025-04-21,AAPL,2025-04-21,3
3,Apple (AAPL) Earnings Preview: Can the Stock R...,Apple (AAPL) Earnings Preview: Can the Stock R...,2025-04-29,https://news.google.com/rss/articles/CBMiiAFBV...,https://www.tastylive.com,tastylive,2025-04-29,AAPL,2025-04-29,4
4,INVESTIGATION ALERT (NASDAQ: AAPL): DiCello Le...,INVESTIGATION ALERT (NASDAQ: AAPL): DiCello Le...,2025-04-24,https://news.google.com/rss/articles/CBMi1gJBV...,https://www.globenewswire.com,GlobeNewswire,2025-04-24,AAPL,2025-04-24,5
...,...,...,...,...,...,...,...,...,...,...
98,"Tech, auto shares gain as Trump floats more ta...","Tech, auto shares gain as Trump floats more ta...",2025-04-14,https://news.google.com/rss/articles/CBMiogFBV...,https://www.reuters.com,Reuters,2025-04-14,AAPL,2025-04-14,99
99,Apple (AAPL) PT Lowered to $230 at Raymond Jam...,Apple (AAPL) PT Lowered to $230 at Raymond Jam...,2025-04-29,https://news.google.com/rss/articles/CBMi3wFBV...,https://www.streetinsider.com,StreetInsider,2025-04-29,AAPL,2025-04-29,100
100,Apple shares fall as Cook says 'very difficult...,Apple shares fall as Cook says 'very difficult...,2025-05-01,https://news.google.com/rss/articles/CBMifEFVX...,https://www.cnbc.com,CNBC,2025-05-01,AAPL,2025-05-01,104
101,"Final Trade: AAPL, NKE, XLP - CNBC","Final Trade: AAPL, NKE, XLP CNBC",2025-05-07,https://news.google.com/rss/articles/CBMidkFVX...,https://www.cnbc.com,CNBC,2025-05-07,AAPL,2025-05-07,128


In [44]:
def groupby_all_articles(ticker:str, start_date:str, end_date:str)->pd.DataFrame:
    df_articles = articles_fetch_data(ticker, start_date, end_date)
    df_articles = df_articles.groupby(['stock_code','date']).agg({'headline': 'count', 'sentiment_score': 'mean'}).reset_index()
    df_articles.rename(columns={'headline': 'article_count'}, inplace=True)
    return df_articles


In [45]:
df = groupby_all_articles('AAPL', '2025-04-01', '2025-05-12')
df

,stock_code,date,article_count,sentiment_score
0,AAPL,2025-04-03,1,-0.827100
1,AAPL,2025-04-04,1,-0.381800
2,AAPL,2025-04-10,5,-0.350480
3,AAPL,2025-04-11,5,-0.000640
4,AAPL,2025-04-12,5,0.168180
5,AAPL,2025-04-13,1,-0.421500
6,AAPL,2025-04-14,13,-0.080185
7,AAPL,2025-04-15,8,0.237138
8,AAPL,2025-04-16,1,0.636900
9,AAPL,2025-04-17,5,0.068520


In [48]:
def merge_tables(ticker:str, start_date:str, end_date:str)->pd.DataFrame:
    df_articles = groupby_all_articles(ticker, start_date, end_date)
    df_prices = fetch_prices(ticker, start_date, end_date)
    df_merged = pd.merge(df_prices, df_articles, on=['stock_code','date'], how='left')
    return df_merged

merge_tables = merge_tables('AAPL', '2025-04-01', '2025-06-12')
merge_tables

,Date,Close,High,Low,Open,Volume,runtime,pct_change,date,stock_name,stock_code,unique_key,row_id,article_count,sentiment_score
0,2025-04-01,222.897705,223.387054,218.613315,219.522127,36412700,2025-06-15 12:10:40.398107,NaN,2025-04-01,Apple,AAPL,2025-04-01,1,NaN,NaN
1,2025-04-02,223.596786,224.895087,220.730550,221.030160,35905900,2025-06-15 12:10:40.398107,0.313633,2025-04-02,Apple,AAPL,2025-04-02,2,NaN,NaN
2,2025-04-03,202.923904,207.218276,200.986443,205.270818,103419000,2025-06-15 12:10:40.398107,-9.245608,2025-04-03,Apple,AAPL,2025-04-03,3,1.0,-0.827100
3,2025-04-04,188.133301,199.618240,187.094654,193.636079,125910900,2025-06-15 12:10:40.398107,-7.288744,2025-04-04,Apple,AAPL,2025-04-04,4,1.0,-0.381800
4,2025-04-07,181.222366,193.895735,174.391312,176.967935,160466300,2025-06-15 12:10:40.398107,-3.673424,2025-04-07,Apple,AAPL,2025-04-07,5,NaN,NaN
5,2025-04-08,172.194199,190.090729,168.988411,186.455496,120859500,2025-06-15 12:10:40.398107,-4.981818,2025-04-08,Apple,AAPL,2025-04-08,6,NaN,NaN
6,2025-04-09,198.589584,200.347274,171.664886,171.724805,184395900,2025-06-15 12:10:40.398107,15.328847,2025-04-09,Apple,AAPL,2025-04-09,7,NaN,NaN
7,2025-04-10,190.170624,194.524915,182.760343,188.822401,121880000,2025-06-15 12:10:40.398107,-4.239377,2025-04-10,Apple,AAPL,2025-04-10,8,5.0,-0.350480
8,2025-04-11,197.890488,199.278667,185.816325,185.856281,87435900,2025-06-15 12:10:40.398107,4.059441,2025-04-11,Apple,AAPL,2025-04-11,9,5.0,-0.000640
9,2025-04-14,202.254776,212.661128,200.896557,211.163092,101352900,2025-06-15 12:10:40.398107,2.205406,2025-04-14,Apple,AAPL,2025-04-14,10,13.0,-0.080185


In [1]:
from src.backend.main import main


ModuleNotFoundError: No module named 'data_handeling'